In [1]:
import socket
import subprocess
import pickle
import threading
import numpy as np
import matplotlib.pyplot as plt
import time
import subprocess

class DroneControl_API:
    def send_command(self, command):
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_socket.connect(('localhost', 5555))
        client_socket.send(command.encode())    
        return client_socket
    
    def take_off(self):
        self.send_command('initialize')

    def motor_control(self, front_right_pwm, rear_left_pwm, front_left_pwm, rear_right_pwm, duration):
        self.send_command(f'motor_control {front_right_pwm} {rear_left_pwm} {front_left_pwm} {rear_right_pwm} {duration}')

    def read_imu_data(self):
        client_socket = self.send_command('read_imu_data')
        time_tag, a_x , a_y , a_z , angular_Vel_x , angular_Vel_y , angular_Vel_z  = self.recieved_data(client_socket)
        return time_tag, a_x , a_y , a_z , angular_Vel_x , angular_Vel_y , angular_Vel_z

    def capture_image (self,camera_side):
        if camera_side == 'front':
            client_socket  = self.send_command('render_image front_cam')

        if camera_side == 'right': 
            client_socket  = self.send_command('render_image right_cam')

        if camera_side == 'left':
            client_socket  = self.send_command('render_image left_cam')
        
        image_data = self.recieved_data(client_socket)
        img1d = np.frombuffer(image_data[0], dtype=np.uint8)
        img_rgb = img1d.reshape(image_data[2], image_data[1], 3)
        plt.imshow(img_rgb)

    def recieved_data(self,client_socket):
        # client_socket = self.send_command('recieved_data')
        response = b''
        while True:
            part = client_socket.recv(4096)
            if not part:
                break
            response += part
        client_socket.close()
        if response != b'':
            response = pickle.loads(response)
        return response

In [2]:
class Kalman_Filter:
    def __init__(self):
        self.A = np.array([[1, 1], [0, 1]])
        self.B = np.array([[0.5], [1]])
        self.C = np.array([[1, 0]])
        self.Q = np.array([[1, 0], [0, 1]])
        self.R = np.array([[1]])
        self.P = np.array([[1, 0], [0, 1]])
        self.x = np.array([[0], [0]])

In [3]:
class IMU_Data():
    def __init__(self,api_control):
        self.kalman_filter = Kalman_Filter()
        self.api_control = api_control

    def run(self):
        time_list = [] 
        ax_list = [] 
        ay_list = [] 
        az_list = []
        Inital_Time, *_  = self.api_control.read_imu_data()
        #print(self.api_control.read_imu_data())
        while True:
            sleep.time(0.01)
            time_tag, *_ = self.api_control.read_imu_data()
            # time_list.append(time_tag)
            # ax_list.append(a_x)
            # ay_list.append(a_y)
            # az_list.append(a_z)
            #print(1e-9*(time_tag - Inital_Time))
            time_list.append(1e-9*(time_tag - Inital_Time))
            print(len(time_list))
            if len(time_list) > 1000:
                break
        plt.plot(time_list)
        plt.show()

In [4]:
# Connection to the AirSim Simulator:
cmd_process = subprocess.Popen('cmd', shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
cmd_process.stdin.write(r'cd C:\Users\yaffe\source\repos\AirSim\myenv\Scripts && python API_Commands_Script.py' + '\n')  # Change directory and execute Python script
cmd_process.stdin.flush()

class Drone_Flight:
    def __init__(self):

        # Connect to the AirSim Simulator & Take off:
        self.API_Control = DroneControl_API()
        self.API_Control.take_off()
        time.sleep(3)
        self.running = True
        # Imu Class:
        self.imu_data = IMU_Data(self.API_Control)

    def start(self):
        start_time = time.time
        self.imu_data.run()
        #imu_thread = threading.Thread(target = self.imu_data.run)
        #imu_thread.start()


In [5]:
x = Drone_Flight()
x.start()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36


: 

: 

In [ ]:
# # Image capturing:
# #drone.capture_image('front')
# drone_api.capture_image('right')
# #drone.capture_image('left')
# drone_api.read_imu_data()
# drone_api.motor_control(0.02, 0.025, 0.02, 0.025, 5)